In [1]:
# conda install opencv

In [1]:
import pandas as pd
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [12]:
# import glob
# import cv2

# images = [cv2.imread(file) for file in glob.glob('./data/train/nowildfire/*.jpg')]

Premature end of JPEG file


In [13]:
# len(images)

14500

In [10]:
# images[0]

array([[[182, 190, 189],
        [207, 215, 214],
        [161, 169, 168],
        ...,
        [181, 193, 195],
        [180, 192, 194],
        [179, 191, 193]],

       [[182, 190, 189],
        [206, 214, 213],
        [161, 169, 168],
        ...,
        [181, 193, 195],
        [180, 192, 194],
        [179, 191, 193]],

       [[182, 190, 189],
        [205, 213, 212],
        [160, 168, 167],
        ...,
        [181, 193, 195],
        [179, 191, 193],
        [179, 191, 193]],

       ...,

       [[ 70,  67,  59],
        [ 71,  68,  60],
        [ 69,  66,  58],
        ...,
        [207, 219, 225],
        [204, 216, 222],
        [199, 211, 217]],

       [[ 69,  66,  58],
        [ 72,  69,  61],
        [ 68,  65,  57],
        ...,
        [207, 219, 225],
        [204, 216, 222],
        [199, 211, 217]],

       [[ 68,  65,  57],
        [ 73,  70,  62],
        [ 67,  64,  56],
        ...,
        [208, 220, 226],
        [205, 217, 223],
        [200, 212, 218]]

In [2]:
from tensorflow.keras.utils import image_dataset_from_directory

In [27]:
img_train = image_dataset_from_directory('./data/train/',  
                                         validation_split = 0.25,
                                         image_size = (28, 28),
                                         subset = 'training', 
                                         seed = 42)
img_test = image_dataset_from_directory('./data/train/', 
                                         validation_split = 0.25, 
                                         image_size = (28, 28),
                                         subset = 'validation', 
                                         seed = 42)

Found 30250 files belonging to 2 classes.
Using 22688 files for training.
Found 30250 files belonging to 2 classes.
Using 7562 files for validation.


In [4]:
img_train

<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [28]:
import os

directory = './data/train/nowildfire'

# Get a list of all items (files and directories) in the directory
all_items = os.listdir(directory)

# Filter out only the file names from the list
file_names = [item for item in all_items if os.path.isfile(os.path.join(directory, item))]


In [29]:
from struct import unpack
# from tqdm import tqdm
import os

marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()
    
    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]            
            if len(data)==0:
                break        


bads = []


for img in file_names:
  image = os.path.join(directory, img)
  image = JPEG(image) 
  try:
    image.decode()   
  except:
    bads.append(img)


for name in bads:
  os.remove(os.path.join(directory,name))

In [20]:
bads

['-114.152378,51.027198.jpg']

In [ ]:
images

In [82]:
class_names = img_train.class_names
print(class_names)

['nowildfire', 'wildfire']


In [30]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [31]:
normalized_ds = img_train.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 0.84583336


In [35]:
model = Sequential()

# Make a convolutional layer with 32 filters.
model.add(Conv2D(32, 3, activation='relu', input_shape=(28, 28, 3)))

# MaxPool the results (basically a requirement)
model.add(MaxPooling2D(2))

# Let's add another convolution block
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(2))

# Finally, flatten the output and make a predictions through a dense layer.
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))

In [36]:
model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='rmsprop',
    metrics=['acc']
)

In [37]:

# Fit!
history = model.fit(
    img_train,
    validation_data=(img_test),
    batch_size=128,
    epochs=10
)

Epoch 1/10
709/709 [==============================] - 11s 14ms/step - loss: 0.8152 - acc: 0.8180 - val_loss: 0.4151 - val_acc: 0.8412
Epoch 2/10
709/709 [==============================] - 19s 26ms/step - loss: 0.2909 - acc: 0.8862 - val_loss: 0.2602 - val_acc: 0.9033
Epoch 3/10
709/709 [==============================] - 11s 15ms/step - loss: 0.2758 - acc: 0.8935 - val_loss: 0.2482 - val_acc: 0.9084
Epoch 4/10
709/709 [==============================] - 12s 17ms/step - loss: 0.2740 - acc: 0.8988 - val_loss: 0.2695 - val_acc: 0.8947
Epoch 5/10
709/709 [==============================] - 11s 16ms/step - loss: 0.2610 - acc: 0.9013 - val_loss: 0.3031 - val_acc: 0.8947
Epoch 6/10
709/709 [==============================] - 15s 21ms/step - loss: 0.2498 - acc: 0.9074 - val_loss: 0.2428 - val_acc: 0.9143
Epoch 7/10
709/709 [==============================] - 12s 17ms/step - loss: 0.2464 - acc: 0.9079 - val_loss: 0.2541 - val_acc: 0.9094
Epoch 8/10
709/709 [==============================] - 11s 15ms

In [ ]:
X = img_array

In [39]:
new_img_array = np.array(img_array)

In [40]:
new_img_array

array(<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
      dtype=object)

In [2]:
import io
import os

WIDTH = 28
HEIGHT = 28

all_images = []
for image_path in os.listdir('./data/train/nowildfire/'):
  img = io.imread(image_path , as_grey=True)
  img = img.reshape([WIDTH, HEIGHT, 1])
  all_images.append(img)
X_train = np.array(all_images)



AttributeError: module 'io' has no attribute 'imread'

In [3]:
# pip install tensorflow

In [22]:
# pip install keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist, cifar10


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist, cifar10

import keras
from tensorflow.keras.utils import load_img

In [ ]:
for file in glob.glob('./data/train/nowildfire/*.jpg'):
    img = 